In [ ]:
import pandas as pd
import numpy as np

In [ ]:
book_df_origin = pd.read_pickle('./small_Books.pkl')

In [ ]:
meta_df_origin = pd.read_pickle('./small_meta_Books.pkl')

In [ ]:
book_df = book_df_origin.copy()
meta_df = meta_df_origin.copy()

In [ ]:
print(meta_df.columns)
print(book_df.columns)

In [ ]:
original_title2id_dict = {}
id_num = 0 # start from 0
unique_titles = meta_df['title'].unique().tolist()
for title in unique_titles:
    original_title2id_dict[title] = id_num
    id_num+=1
    

In [ ]:
meta_df['title_idx'] = meta_df['title'].apply(lambda x:original_title2id_dict[x])

# 1. seperate cold-start and warm-start items according to item publish time

In [ ]:
print(book_df.columns, meta_df.columns)

In [ ]:
partial_meta_df = meta_df.copy()[['title','parent_asin','title_idx','publish_time']]

In [ ]:
merged_book_df = pd.merge(left = book_df, right=partial_meta_df,how = 'left',on='parent_asin', )

In [ ]:
print(merged_book_df.columns)

In [ ]:
data = merged_book_df.copy()

In [ ]:
data.columns = ['rating', 'iid', 'uid', 'timestamp', 'title', 'title_idx','publish_time']
data['year'] = data['timestamp'].dt.year

In [ ]:
print(data.head(5))

In [ ]:
cold_publish_data = data[data['publish_time'] >= pd.to_datetime('2022-10-01')]
cold_publish_items = cold_publish_data['title_idx'].unique().tolist()

In [ ]:
item_review_earliest_time = data.groupby('title_idx').agg({'timestamp':'min'})
print(item_review_earliest_time)

In [ ]:
cold_review_data = item_review_earliest_time[item_review_earliest_time['timestamp'] >= pd.to_datetime('2022-10-01')]
cold_review_items = cold_review_data.index.tolist()

In [ ]:
cold_items = set(cold_review_items).intersection(set(cold_publish_items))

# 2.get warm_train, warm_test, cold_test data

In [ ]:
warm_start_time = '2016-01-01'
cold_end_time = '2023-10-01'

In [ ]:
data = merged_book_df.copy()
data.columns = ['rating', 'iid', 'uid', 'timestamp', 'title', 'title_idx','publish_time']
data['year'] = data['timestamp'].dt.year
data = data[(data['timestamp'] >= pd.to_datetime(warm_start_time)) & (data['timestamp'] <= pd.to_datetime(cold_end_time))]

In [ ]:
data['month'] = data['timestamp'].dt.month + data['timestamp'].dt.year * 12
data['month'] = data['month'] - data['month'].min()

In [ ]:
# delete cold data' interactions
test_months = list(range(data['month'].max()-12+1,data['month'].max() +1 )) 
cold_data_test = cold_data[cold_data['month'].isin(test_months)]
cold_items_test = cold_data_test['title_idx'].unique().tolist()

In [ ]:
preserve_warm_data = uncold_data
cnt_warm = preserve_warm_data['iid'].value_counts() 

In [ ]:
# process only warm_data
preserve_warm_data_test = preserve_warm_data[preserve_warm_data['month'].isin(test_months)]
preserve_warm_data_train = preserve_warm_data[~preserve_warm_data['month'].isin(test_months)]

In [ ]:
preserve_warm_data_train['env'] = 0 # warm train
preserve_warm_data_test['env'] = 1
# the fake cold data denote items published before '2022-10-31',  
# which means their text information may be collected in LLAMA-2's pretraining stage.
# However, these items were not used in our final paper.
# fake_cold_data_test['env'] = 2,fake_cold_data_test
cold_data_test['env'] = 3
all_data = pd.concat([preserve_warm_data_train,preserve_warm_data_test,cold_data_test])


In [ ]:
def filter_users(all_data):
    user_cnt = all_data['uid'].value_counts()
    warm_users = user_cnt[user_cnt >= 20].index.tolist()
    all_data = all_data[all_data['uid'].isin(warm_users)]
    return all_data

In [ ]:
# filter items, preserve all items with interaction >= 20.
def filer_items(all_data):
    item_cnt = all_data['iid'].value_counts()
    filer_items = item_cnt[item_cnt >= 20].index.tolist()
    all_data = all_data[all_data['iid'].isin(filer_items)]
    train_warm_cnt = all_data[all_data['env'] == 0]['iid'].value_counts()
    delete_items = train_warm_cnt[train_warm_cnt < 20].index.unique().tolist()
    all_data = all_data[~all_data['iid'].isin(delete_items)]
    return all_data

In [ ]:
print(all_data.shape[0])
for i in range(5):
    all_data = filter_users(all_data)
    all_data = filer_items(all_data)

In [ ]:
print(all_data.shape[0])
print(all_data[all_data['env'] == 0].shape[0]) # warm_train
print(all_data[all_data['env'] == 1].shape[0]) # warm_test
print(all_data[all_data['env'] == 3].shape[0]) # cold

print(all_data[all_data['env'] == 0]['iid'].value_counts())
print(all_data[all_data['env'] == 1]['iid'].value_counts())
print(all_data[all_data['env'] == 3]['iid'].value_counts())

# 3.save data for TALLRec and CF model

In [ ]:
# parent_asin to iid
parent_asin2iid_dict = {}
num = 0
for parent_asin in all_data['iid'].unique().tolist():
    parent_asin2iid_dict[parent_asin] = num
    num += 1
all_data['iid'] = all_data['iid'].apply(lambda x: parent_asin2iid_dict[x])

In [ ]:
print(len(parent_asin2iid_dict))

In [ ]:
# parent_asin to iid
user_id2uid_dict = {}
num = 0
for user_id in all_data['uid'].unique().tolist():
    user_id2uid_dict[user_id] = num
    num += 1
all_data['uid'] = all_data['uid'].apply(lambda x: user_id2uid_dict[x])

In [ ]:
print(len(user_id2uid_dict))

In [ ]:
print(all_data.columns)

In [ ]:
print(all_data.isnull().any())

In [ ]:
print(all_data['publish_time'].isnull().sum())

In [ ]:
all_data.sort_values(by = 'timestamp', inplace=True, ascending=True)
all_data['label'] = all_data['rating'].apply(lambda x: 1 if x>=4 else 0)
all_data.drop_duplicates(inplace=True)
print(all_data.shape) 

In [ ]:
u_inter_all = all_data.groupby('uid').agg({'iid':list, 'label':list, 'title':list, 'timestamp':list, 'env':list})
print(u_inter_all.head(1))

In [ ]:
import copy
# get  [(seq0, label0), (seq1,label1)] for one user
def deal_with_each_u(x,u):
    items = np.array(x.iid)
    labels = np.array(x.label)
    titles = np.array(x.title)
    timestamp = np.array(x.timestamp)
    env = np.array(x.env)
    his = [0] # adding a '0' by default
    his_title = ['']
    results = []
    for i in range(items.shape[0]):
        results.append((u, items[i], timestamp[i], np.array(his), copy.copy(his_title),titles[i], labels[i],env[i]))
        # training data
        if labels[i] > 0: # positive 
            his.append(items[i])
            his_title.append(titles[i])
    return results

In [ ]:
results = []
for u in u_inter_all.index:
    results.extend(deal_with_each_u(u_inter_all.loc[u],u))

In [ ]:
u_, i_, time_, label_, his_, his_title, title_,env_ = [],[],[],[],[],[],[],[]
for re_ in results:
    # if len(re_[3]) < 11:
    #     continue
    # else:
        u_.append(re_[0])
        i_.append(re_[1])
        time_.append(re_[2])
        his_.append(re_[3][-15:])
        his_title.append(re_[4][-15:])
        title_.append(re_[5])
        label_.append(re_[6])
        env_.append(re_[7])

In [ ]:
all_data = pd.DataFrame({"uid":u_,'iid':i_,'label':label_, 'timestamp': time_ , 'his':his_,'his_title':his_title,'title':title_,'env':env_})

In [ ]:
print(all_data.shape)
all_data.sort_values(by = 'timestamp', ascending=True,inplace=True)

In [ ]:
train_data = all_data[all_data['env'] == 0]
warm_test = all_data[all_data['env'] == 1]
cold_test = all_data[all_data['env'] == 3]
print('warm train inter num', train_data.shape[0]) # warm_train
print('warm evaluate inter num', warm_test.shape[0]) # warm_test
print('cold evaluate inter num', cold_test.shape[0]) # cold (cold_test)
mix_test = pd.concat((warm_test, cold_test))
unique_iids = mix_test['iid'].unique().tolist()
valid_data = []
test_data = []
for iid in unique_iids:
    iid_data = mix_test.loc[mix_test['iid'] == iid]
    split_index = len(iid_data) // 2  
    valid_data.append(iid_data.iloc[:split_index])
    test_data.append(iid_data.iloc[split_index:])
valid_all = pd.concat(valid_data)
test_all = pd.concat(test_data)
warm_valid = valid_all[valid_all['env'] == 1]
cold_valid = valid_all[valid_all['env'] == 3]
warm_test = test_all[test_all['env'] == 1]
cold_test = test_all[test_all['env'] == 3]

In [ ]:
train_data.to_pickle('./train0.pkl')  # train_warm
valid_all.to_pickle('./valid0.pkl') # valid_mix
test_all.to_pickle('./test0.pkl') # valid_mix
warm_valid.to_pickle('./valid1.pkl') # valid_warm
cold_valid.to_pickle('./valid2.pkl') # valid_cold
warm_test.to_pickle('./test1.pkl') # test_warm
cold_test.to_pickle('./test2.pkl')  # test_cold